In [1]:
import numpy as np
import pandas as pd
import string
import re
import nltk
import json

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df1 = pd.read_csv('Patient-Therapist.csv')
df1.head()

,prompt,completion
0,Me and the father of my child have been dating...,It sounds like you are feeling very alone and ...
1,I can't seem to feel any emotion except anxiet...,It is possible that you are experiencing sympt...
2,why do we allow one or few bad experiences whe...,There are a few possible explanations for why ...
3,"I have major depression, severe, PTSD, anxiety...",It is understandable that you are feeling upse...
4,do i need a therapist,There is no one-size-fits-all answer to this q...


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522 entries, 0 to 1521
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prompt      1522 non-null   object
 1   completion  1522 non-null   object
dtypes: object(2)
memory usage: 23.9+ KB


In [5]:
with open('intents.json', 'r') as file:
    intent_data = json.load(file)
df2 = pd.DataFrame(intent_data['intents'])
df2.head()

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."


In [6]:
df2 = df2.drop(columns=['tag'])
df2

,patterns,responses
0,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,[Good morning],[Good morning. I hope you had a good night's s...
2,[Good afternoon],[Good afternoon. How is your day going?]
3,[Good evening],[Good evening. How has your day been?]
4,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...
75,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...
76,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [7]:
# Rename 'patterns' to 'prompt' and 'responses' to 'completion'
df2.rename(columns={'patterns': 'prompt', 'responses': 'completion'}, inplace=True)
df2.head()

,prompt,completion
0,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,[Good morning],[Good morning. I hope you had a good night's s...
2,[Good afternoon],[Good afternoon. How is your day going?]
3,[Good evening],[Good evening. How has your day been?]
4,[Good night],"[Good night. Get some proper sleep, Good night..."


In [8]:
# Extract text inside square brackets in both columns, handling cases where brackets may not exist
df2['prompt'] = df2['prompt'].astype(str).str.extract(r'\[([^\]]*)\]').replace("'", "", regex=True)
df2['completion'] = df2['completion'].astype(str).str.extract(r'\[([^\]]*)\]').replace("'", "", regex=True)
df2.head()

,prompt,completion
0,"Hi, Hey, Is anyone there?, Hi there, Hello, He...",Hello there. Tell me how are you feeling today...
1,Good morning,"""Good morning. I hope you had a good nights sl..."
2,Good afternoon,Good afternoon. How is your day going?
3,Good evening,Good evening. How has your day been?
4,Good night,"Good night. Get some proper sleep, Good night...."


In [9]:
# Merge DataFrames based on the 'tag' column
df = pd.concat([df1, df2], ignore_index=True)
df

,prompt,completion
0,Me and the father of my child have been dating...,It sounds like you are feeling very alone and ...
1,I can't seem to feel any emotion except anxiet...,It is possible that you are experiencing sympt...
2,why do we allow one or few bad experiences whe...,There are a few possible explanations for why ...
3,"I have major depression, severe, PTSD, anxiety...",It is understandable that you are feeling upse...
4,do i need a therapist,There is no one-size-fits-all answer to this q...
...,...,...
1597,"""What do I do if Im worried about my mental he...",The most important thing is to talk to someone...
1598,"""How do I know if Im unwell?""","If your beliefs , thoughts , feelings or behav..."
1599,How can I maintain social connections? What if...,"""A lot of people are alone right now, but we d..."
1600,"""Whats the difference between anxiety and stre...","""Stress and anxiety are often used interchange..."


In [10]:
from nltk.tokenize import word_tokenize
df["prompt"]=df["prompt"].str.lower()

In [11]:
text=df["prompt"][1600]

In [13]:
from nltk.tokenize import RegexpTokenizer
text = re.sub(r'[^\w\s]', '', text)
words = word_tokenize(text)
words

['whats', 'the', 'difference', 'between', 'anxiety', 'and', 'stress']

In [15]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
text=" ".join([wnl.lemmatize(t) for t in text.split() ])
text

'whats the difference between anxiety and stress'

In [16]:
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

In [17]:
def data_prep(text):
  text=text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text=" ".join(t for t in text.split() if t not in stopwords)
  return text

In [18]:
print(data_prep(text))

whats difference anxiety stress


In [19]:
df["prompt"]=df.loc[:,"prompt"].apply(data_prep)

In [20]:
df["prompt"]

0       father child dating year big argument first wa...
1              cant seem feel emotion except anxiety even
2       allow one bad experiences felt worthless dicta...
3       major depression severe ptsd anxiety disorder ...
4                                          need therapist
                              ...                        
1597                             im worried mental health
1598                                       know im unwell
1599              maintain social connections feel lonely
1600                      whats difference anxiety stress
1601    whats difference sadness depression difference...
Name: prompt, Length: 1602, dtype: object

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf_train = tf.fit_transform(df["prompt"])

In [25]:
# Get feature names from the vocabulary
feature_names = tf.get_feature_names_out()
# Create a DataFrame with the transformed data and feature names
df_check = pd.DataFrame(tf_train.toarray(), columns=feature_names)

In [26]:
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()
df["completion_code"] = le.fit_transform(df["completion"])

df["completion_code"]

0        561
1        482
2        869
3        498
4       1314
        ... 
1597     846
1598     367
1599       5
1600      34
1601      30
Name: completion_code, Length: 1602, dtype: int64

In [27]:
df.head()

,prompt,completion,completion_code
0,father child dating year big argument first wa...,It sounds like you are feeling very alone and ...,561
1,cant seem feel emotion except anxiety even,It is possible that you are experiencing sympt...,482
2,allow one bad experiences felt worthless dicta...,There are a few possible explanations for why ...,869
3,major depression severe ptsd anxiety disorder ...,It is understandable that you are feeling upse...,498
4,need therapist,There is no one-size-fits-all answer to this q...,1314


In [28]:
from sklearn.naive_bayes import MultinomialNB

mn=MultinomialNB()
mn.fit(df_check,df["completion_code"])

MultinomialNB()

In [29]:
test=[
    "I can't seem to feel any emotion except anxiety, not even for myself."
]

testing = tf.transform(test)

In [30]:
mn.predict(testing)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


array([482])

In [31]:
completion = df['completion'].unique()
completion_list = completion.tolist()

completion_code = df['completion_code'].unique()
completion_code_list = completion_code.tolist()

In [32]:
completion_code_list.index(482)

1

In [33]:
completion[1]

'It is possible that you are experiencing symptoms of an anxiety disorder. Anxiety disorders are the most common type of mental illness, and they can make it difficult to cope with everyday life. Symptoms of anxiety can include feeling restless, irritable, and easily fatigued; having difficulty concentrating; and experiencing muscle tension and sleep disturbances. If you are experiencing these symptoms, it is important to talk to a mental health professional.'

In [34]:
def prompt(sentences):
  prompts = tf.transform(sentences)
  predicts = mn.predict(prompts)

  completion = df['completion'].unique()
  completion_list = completion.tolist()

  completion_code = df['completion_code'].unique()
  completion_code_list = completion_code.tolist()



  for completion_index in predicts:
    ans = completion_code_list.index(completion_index)
    print(completion[ans])

In [35]:
prompt(["I can't seem to feel any emotion except anxiety, not even for myself."])

It is possible that you are experiencing symptoms of an anxiety disorder. Anxiety disorders are the most common type of mental illness, and they can make it difficult to cope with everyday life. Symptoms of anxiety can include feeling restless, irritable, and easily fatigued; having difficulty concentrating; and experiencing muscle tension and sleep disturbances. If you are experiencing these symptoms, it is important to talk to a mental health professional.


In [36]:
prompt(["do I need a therapist"])

There is no one-size-fits-all answer to this question, as the need for a therapist depends on a variety of individual factors. However, if you are struggling with mental health issues or experiencing difficulty coping with life stressors, seeking professional help may be beneficial. A therapist can provide support and guidance as you work through your challenges, and can also offer coping and problem-solving strategies that can help you improve your overall well-being.


In [41]:
prompt(["Good morning"])

"Good morning. I hope you had a good nights sleep. How are you feeling today? "


In [46]:
prompt(["Good afternoon"])

Good afternoon. How is your day going?
